In [1]:
!pip install opencv-python
!pip install pyttsx3

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import cv2
import numpy as np
import pyttsx3
from concurrent.futures import ThreadPoolExecutor

engine = pyttsx3.init()
attentiveness_score = 100

def process_frame(frame):
    global attentiveness_score

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        eyes = eye_cascade.detectMultiScale(roi_gray)
        attentiveness_score = 100 if len(eyes) > 0 else 0

        cv2.putText(frame, f"Attentiveness: {attentiveness_score}%", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        if attentiveness_score == 0:
            cv2.putText(frame, "Sleeping", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            engine.say("Attention! The student is sleeping.")
            engine.runAndWait()

        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    cv2.imshow('Video', frame)

video_capture = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
executor = ThreadPoolExecutor(max_workers=2)

while True:
    ret, frame = video_capture.read()

    if not ret or frame is None:
        print("Error: Could not read frame from the webcam.")
        break

    executor.submit(process_frame, frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
